## 딥러닝

#### 인공 신경망의 한계와 딥러닝 출현

- 오늘날 인공 신경망에서 사용하는 구조는 Frank Rosenblatt가 1957년에 고안한 **퍼셉트론**이라는 선형 분류기임

- 퍼셉트론은 다수의 신호(x1, x2)를 입력으로 받아 하나의 신호(y)를 출력하는데, 이 신호를 입력으로 받아 **0/1** 정보를 전달하며 작동함

<img src="https://images.velog.io/images/kyj93790/post/307d968e-2f2c-4606-b5ca-e43f2ff33343/image.png" width="280px">

- XOR 연산과 같이 **비선형적**으로 분리되는 데이터에 대해서도 학습이 가능하도록 **다층 퍼셉트론**이 고안됨

- 입력층과 출력층 사이에 여러 개의 은닉층을 두는 방법 : **심층 신경망**

---

## 구성 요소

#### 구조

- 입력층(input layer)

  - 데이터를 받아들이는 층
<br><br>
- 은닉층(hidden layer)

  - 모든 입력 노드로부터 입력 값을 받아 **weighted sum**을 계산함

  - 이 값을 **활성화 함수**에 적용하여 출력층에 전달함
<br><br>
- 출력층(output layer)

  - 신경망의 최종 결과값이 포함된 층

<img src="https://www.i2tutorials.com/wp-content/media/2019/05/Hidden-layrs-1-i2tutorials.jpg" width="450px">

- 가중치(weight)

  - 노드와 노드 간의 연결 강도
<br><br>
- 편향(bias)

  - 활성화 함수를 거쳐 나온 출력값을 조절하는 역할
<br><br>
- 가중합(weighted sum)

  - 신호*가중치 값을 모두 합한 것
<br><br>
- 함수

  - 활성화 함수 : 신호를 입력받아 이를 적절히 처리하는 함수
  
  - 손실 함수 : 함수의 결과값과 실제값 사이 오차를 측정하는 함수

##### - 활성화 함수

- Sigmoid

  - 0~1 사이의 값으로 변환함

- Hyperbolic Tangent

  - -1~1 사이의 값으로 변환함

- ReLU

  - 입력(x)이 음수일 때는 0, 양수일 때는 x로 출력함
  
  - Gradient Vanishing 문제가 발생하지 않아, 일반적으로 은닉층에 자주 사용됨

- Softmax

  - 0~1 사이의 값으로 정규화하여, 출력 값들의 총합이 1이 되도록 함
  
  - 출력 노드에서 많이 사용됨

<img src="https://mblogthumb-phinf.pstatic.net/MjAyMDAyMjVfMTQ3/MDAxNTgyNjA5NDY3MTY3.228bUv_5mrol1w7X0NiFMD1UNru9zyf3yIJGcON-An0g.3Kzynlja9y_F9yTfANl937elQAK1pTGoJ_al7Om7TYsg.PNG.handuelly/image.png?type=w800" width="400px">

In [7]:
import torch
import torch.nn as nn

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_feature, n_hidden) # 은닉층
        self.relu = nn.ReLU(inplace=True) # 은닉층을 위한 활성화 함수
        self.out = nn.Linear(n_hidden, n_output) # 출력층
        self.softmax = nn.Softmax(dim=n_output) # 출력층을 위한 활성화 함수


    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.out(x)
        x = self.softmax(x)
        return x

##### - 손실 함수

- 학습을 통해 얻은 추정치가 실제 정답과 얼마나 차이가 있는지 평가하는 지표

- Mean Squared Error
  
  - 회귀 문제에서 주로 사용됨

In [28]:
loss_fn = nn.MSELoss(reduction='sum') # MSE
y_pred = model(X_data)
loss = loss_fn(y_pred, Y_data)

- Cross Entropy Error

  - 분류 문제에서 주로 사용됨

In [47]:
loss_fn = nn.CrossEntropyLoss() # CrossEntropy
input = torch.randn(5, 6, requires_grad=True)
target = torch.empty(5, dtype=torch.long).random_(5)
output = loss_fn(input, target)
output.backward()

---

## 학습

#### 순전파(Feed Forward)

- 네트워크에 학습 데이터가 들어올 때 발생함

- 이전 레이어의 뉴런에서 받은 정보에 **weighted sum**과 **활성화 함수**를 적용하여, 다음 레이어의 뉴런으로 전송하는 방식

- 데이터가 모든 레이어를 통과하고 모든 뉴런이 계산을 완료하면, 최종 예측값은 출력층에 도달함

<img src="https://editor.analyticsvidhya.com/uploads/88777Slide3.PNG" width="600px">

#### 역전파(Back Propagation)

- **손실함수**를 통해 네트워크의 예측값과 정답값의 차이를 계산함

- 해당 오차가 0이 되는 방향으로 각 뉴런의 **가중치**(weight)를 조정함

  - *오차는 은닉층의 모든 뉴런으로 전파되지만, 각 뉴런이 원래 출력에 기여한 상대적 기여도(weight)에 따라 값이 달라짐*

  - *오차를 각 뉴런의 weight로 미분한 후, 기존 weight 값에서 빼는 방식으로 업데이트함*

- 다시 순전파의 가중치 값으로 사용함

<img src="https://editor.analyticsvidhya.com/uploads/18870backprop2.png" width="600px">

---

## 문제점

- 딥러닝의 핵심은 활성화 함수가 적용된 **은닉층**을 여러 개 쌓아, **비선형 영역**을 표현하는 것

- 따라서, 은닉층이 많을수록 학습 데이터의 분류가 잘 될 수 있음

##### 은닉층이 많을수록 발생할 수 있는 문제

- 과적합 문제

  - 학습 데이터에 과하게 학습되여, 실제 데이터에 대한 오차가 증가하는 현상
<br><br>
- 기울기 소멸 문제

  - 은닉층에서 전달될 때 기울기가 0에 가까워져, 학습이 진행되지 않는 현상

##### 손실함수 값을 최소화하는 global minimum을 찾기 위한 경사 하강법의 문제

- 배치 경사 하강법

  - **전체 데이터셋**을 사용하여 gradient를 구하고, 파라미터를 업데이트 함

  - 학습이 오래 걸림
<br><br>
- 확률적 경사 하강법

  - **임의로 선택한 데이터**를 사용하여 gradient를 구하고, 파라미터를 업데이트 함

  - 학습 속도가 빠르지만, 파라미터 변경 폭이 불안정하고 정확도가 낮을 수 있음
<br><br>
- 미니 배치 경사 하강법

  - 전체 데이터셋을 **미니 배치**로 나눔

  - 각 미니 배치에 대해 gradient를 구하고, 그것들의 평균 gradient를 사용하여 파라미터를 업데이트 함
  
  - 학습 속도가 빠르고, 파라미터 변경 폭이 안정적임

---

## 사용 시 이점

- 딥러닝에서는 도메인 지식과 인간의 개입이 필요한 **특성 추출** 과정을 알고리즘에 통합시켰음

    - 특성 추출 : 데이터의 특징을 찾아내고, 이를 토대로 데이터를 벡터로 변환하는 작업

---

## 딥러닝 알고리즘

- 심층 신경망을 사용하는 딥러닝 알고리즘은 목적에 따라 분류될 수 있음

#### 심층 신경망(Deep Neural Network)

- 입력층과 출력층 사이에 다수의 은닉층을 포함하는 네트워크

- 다수의 은닉층을 통해 다양한 비선형적 관계를 학습할 수 있음

#### 합성곱 신경망(Convolutional Neural Network)

- 합성곱층과 풀링층을 포함하는 네트워크

- 이미지의 공간 정보를 유지하면서 이미지의 특징을 추출하고 학습할 수 있음

#### 순환 신경망(Recurrent Neural Network)

- 시간의 흐름에 따라 변화하는 시계열 데이터를 학습하기 위한 네트워크

- Recurrent : 현재 결과가 이전 결과와 연관이 있음을 의미함

#### 제한된 볼츠만 머신(Restricted Boltzmann Machine), 심층 신뢰 신경망(Deep Belief Network)